In [1]:
# Задача определить абонентов (msisdn), которые являются одной и той же Персоной (человеком).
import csv
import pandas as pd
import numpy as np
pd.options.display.max_columns = 1000

In [35]:
# Информация о перемещении случайного набора абонентов за период 23-25 мая 2013 года
# в котором точно присутствуют номера телефонов, принадлежащие одному и тому же человеку.
ints_a=["lac","cid","msisdn","event_type","tstamp"]
floats_a=["imei","max_dist","long","lat","start_angle","end_angle"]
df_a = pd.read_csv('../input/02_Data_test.csv', sep=";")
for col in ints_a:
    df_a[col] = df_a[col].apply(lambda s: int(s))
for col in floats_a:
    df_a[col] = df_a[col].apply(lambda s: float(s))
df_a[df_a.index < 3]

lac
cid
msisdn
event_type
tstamp


,lac,cid,msisdn,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle
0,5029,40798,158529599791,3.531111e+14,0,1369252800974,37.9305,55.7815,2101.0,OUTDOOR,220.0,280.0
1,7782,56870,158520145943,1.292900e+13,5,1369252801396,37.4239,55.7426,2.0,OUTDOOR,45.0,105.0
2,7794,32226,158521798391,3.519940e+14,5,1369252802013,37.7034,55.5803,1125.0,OUTDOOR,0.0,60.0


In [3]:
# обучающая выборка: набор пар номеров телефонов (msisdn)
# каждая из которых с большой долей вероятности принадлежит одной Персоне.
df_b = pd.read_excel('../input/01_RU_fakty.xlsx', sep=";", header=None, sheet='Лист 1')
df_b[df_b.index < 3]

,0,1
0,158530067653,158528861653
1,158510912201,158528852857
2,158530063945,158528861599


In [25]:
# Первые 8 цифр imei называются TAC и определяют устройство, которым пользуется абонент.
# Сокращенный справочник TAC (файл 03_устройства) состоит из полей.
cols_c=["tac","vendor","platform","type"]
df_c = pd.read_csv('../input/03_RU_ustroystva.csv',sep=',\s*',skiprows=1,names=cols_c)
for col in cols_c:
    df_c[col] = df_c[col].apply(lambda s: s.replace("\"",""))
df_c[df_c.index < 3]

C:\SOFFFT\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,tac,vendor,platform,type
0,00100429,PROTO,Proprietary,Phone
1,00100430,PROTO,Proprietary,Phone
2,00100431,PROTO,Proprietary,Phone


In [26]:
# Каждая регистрации на базовой станции имеет свой тип.
# Справочник типов регистраций с расшифровкой в файле 04_event_type. 
cols_d=["number","label","description"]
df_d = pd.read_excel('../input/04_event_type.xlsx', sep=";", skiprows=1, names=cols_d, sheet='Лист 1')
df_d[df_d.index < 3]

,number,label,description
0,1,imsi_att_с,IMSI Attach. Включение телефона в той же облас...
1,2,par_c,Изменение местоположения абонента во время раз...
2,3,delete_c,Абонент удалён из VLR.


In [31]:
df_train = df_b.rename(columns={0:"v_tel_0",1:"v_tel_1"})
df_train = df_train.merge(df_a, left_on="v_tel_0", right_on="msisdn", how="left")
df_train[df_train.index < 10]

,v_tel_0,v_tel_1,lac,cid,msisdn,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle
0,158530067653,158528861653,7782,48428,158530067653,1.303401e+13,0,1369281266860,37.4625,55.7214,50,OUTDOOR,170.0,230.0
1,158530067653,158528861653,7752,30479,158530067653,1.303401e+13,0,1369281821169,37.5049,55.7309,1246,OUTDOOR,290.0,350.0
2,158530067653,158528861653,7742,58364,158530067653,1.303401e+13,0,1369281894211,37.4842,55.7393,50,OUTDOOR,86.5,133.5
3,158530067653,158528861653,7742,26211,158530067653,1.303401e+13,1,1369282509891,37.5392,55.7668,628,OUTDOOR,290.0,350.0
4,158530067653,158528861653,7799,20136,158530067653,1.303401e+13,0,1369283494251,37.5876,55.7912,1108,OUTDOOR,330.0,30.0
5,158530067653,158528861653,7799,9705,158530067653,1.303401e+13,1,1369294037018,37.5788,55.7964,253,INDOOR,0.0,360.0
6,158530067653,158528861653,7799,9705,158530067653,1.303401e+13,10,1369297463477,37.5788,55.7964,253,INDOOR,0.0,360.0
7,158530067653,158528861653,7799,7173,158530067653,1.303401e+13,2,1369303333463,37.5730,55.8007,449,OUTDOOR,85.0,145.0
8,158530067653,158528861653,7799,9705,158530067653,1.303401e+13,8,1369310811017,37.5788,55.7964,253,INDOOR,0.0,360.0
9,158530067653,158528861653,7799,21582,158530067653,1.303401e+13,10,1369317711384,37.5777,55.7975,109,OUTDOOR,90.0,150.0
